In [ ]:
!pip install plotly

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
#import numpy as np
#from datetime import datetime
#import plotly.express as px

In [ ]:
#Deaths = pd.read_csv("drive/Shareddrives/Datasets/covid_deaths.csv")
#Cases = pd.read_csv("drive/Shareddrives/Datasets/covid_confirmed.csv")
#Population = pd.read_csv("drive/Shareddrives/Datasets/covid_county_population.csv")
Deaths = pd.read_csv("data/covid_deaths.csv")
Cases = pd.read_csv("data/covid_confirmed.csv")
Population = pd.read_csv("data/covid_county_population.csv")

Preparing for safe merge. Merge County and Population. Cases dataset is converted to long format and merged with the Death_Population_long dataset. Read in Superset_Long csv result.

In [ ]:
Deaths = Deaths[Deaths['County Name'] != 'Statewide Unallocated']
Cases = Cases[Cases['County Name'] != 'Statewide Unallocated']
Population = Population[Population['County Name'] != 'Statewide Unallocated']
Population = Population.drop(columns = ['County Name'])

In [ ]:
Death_Population = pd.merge(Deaths, Population, on = ['countyFIPS', 'State'])
Death_Population_long = pd.melt(Death_Population,id_vars=['countyFIPS','State','County Name','StateFIPS','population'],
             var_name='Date', value_name='Deaths')

In [ ]:
Cases_long = pd.melt(Cases,id_vars=['countyFIPS','State','County Name','StateFIPS'], var_name='Date', value_name='Cases')
merge_dcp_long = pd.merge(Death_Population_long,Cases_long, on=['countyFIPS','State','County Name','StateFIPS', 'Date'])

In [ ]:
Superset_Long = pd.read_csv("drive/Shareddrives/Datasets/superset_long.csv")

Reading superset long data set for usage (Refer to Global_Comparison.ipynb for details on its creation.) Converting Date column to Date datatype and fill the NaN with zero '0'. The data is stored in new DataFrame 'updated'.

In [ ]:
merge_dcp_long = pd.merge(Death_Population_long,Cases_long, on=['countyFIPS','State','County Name','StateFIPS', 'Date'])

In [ ]:
date = pd.DataFrame(pd.to_datetime(merge_dcp_long['Date']))
no_date = pd.DataFrame(merge_dcp_long.drop(columns=['Date']))
updated = pd.concat([date.reset_index(drop=True),no_date.reset_index(drop=True)], axis=1).fillna(0)
updated = updated.sort_values(by=['State', 'Date'])

In [ ]:
updated['New_Cases_Per_Day'] = updated.groupby('State').Cases.diff()
updated['New_Deaths_Per_Day'] = updated.groupby('State').Deaths.diff()

In [ ]:
weeklyData = updated.groupby([pd.Grouper(key='Date', freq='1w'),'State'])['New_Cases_Per_Day','New_Deaths_Per_Day'].agg({'New_Cases_Per_Day':'mean',
                                                                                                'New_Deaths_Per_Day' :'mean'}).reset_index().round(decimals=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


##Statistical data for new cases, deaths, and population in Pennslyvania, per day.

In [ ]:
weeklyDataPA = weeklyData.loc[weeklyData['State'] == 'PA']

Philadelphia_weeklyCases_mean = weeklyDataPA['New_Cases_Per_Day'].mean()
Philadelphia_weeklyCases_median = weeklyDataPA['New_Cases_Per_Day'].median()
Philadelphia_weeklyCases_mode = weeklyDataPA['New_Cases_Per_Day'].mode()

print('Weekly Cases Mean: ', Philadelphia_weeklyCases_mean)
print('Weekly Cases Median: ', Philadelphia_weeklyCases_median)
print('Weekly Cases Mode: ', Philadelphia_weeklyCases_mode)

Weekly Cases Mean:  1.1927710843373494
Weekly Cases Median:  1.0
Weekly Cases Mode:  0    0.0
dtype: float64


In [ ]:
Philadelphia_weeklyDeaths_mean = weeklyDataPA['New_Deaths_Per_Day'].mean()
Philadelphia_weeklyDeaths_median = weeklyDataPA['New_Deaths_Per_Day'].median()
Philadelphia_weeklyDeaths_mode = weeklyDataPA['New_Deaths_Per_Day'].mode()

print('Weekly Deaths Mean: ', Philadelphia_weeklyDeaths_mean)
print('Weekly Deaths Median: ', Philadelphia_weeklyDeaths_median)
print('Weekly Deaths Mode: ', Philadelphia_weeklyDeaths_mode)

Weekly Deaths Mean:  0.0
Weekly Deaths Median:  0.0
Weekly Deaths Mode:  0    0.0
dtype: float64


##Comparing Pennslyvania to North Carolina

In [ ]:
weeklyDataNC = weeklyData.loc[weeklyData['State'] == 'NC']

NCarolina_weeklyCases_mean = weeklyDataNC['New_Cases_Per_Day'].mean()
NCarolina_weeklyCases_median = weeklyDataNC['New_Cases_Per_Day'].median()
NCarolina_weeklyCases_mode = weeklyDataNC['New_Cases_Per_Day'].mode()

print('Weekly Cases Mean: ', NCarolina_weeklyCases_mean)
print('Weekly Cases Median: ', NCarolina_weeklyCases_median)
print('Weekly Cases Mode: ', NCarolina_weeklyCases_mode)

Weekly Cases Mean:  0.012048192771084338
Weekly Cases Median:  0.0
Weekly Cases Mode:  0    0.0
dtype: float64


In [ ]:
NCarolina_weeklyDeaths_mean = weeklyDataNC['New_Deaths_Per_Day'].mean()
NCarolina_weeklyDeaths_median = weeklyDataNC['New_Deaths_Per_Day'].median()
NCarolina_weeklyDeaths_mode = weeklyDataNC['New_Deaths_Per_Day'].mode()

print('Weekly Deaths Mean: ', NCarolina_weeklyDeaths_mean)
print('Weekly Deaths Median: ', NCarolina_weeklyDeaths_median)
print('Weekly Deaths Mode: ', NCarolina_weeklyDeaths_mode)

Weekly Deaths Mean:  0.0
Weekly Deaths Median:  0.0
Weekly Deaths Mode:  0    0.0
dtype: float64
